# Deep Learning - MCH1
FS23, Manuel Schwarz

In [ ]:
import os
import copy
import time
import torch
import wandb
import torchvision
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
from tqdm import tqdm 
from datetime import datetime
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device: ', device)

# sound
import time
import winsound
import datetime

# Schritt 1: Auswahl Task / Datensatz

<div class="alert alert-block alert-info">
    
1. Mache Dir Gedanken, mit welchen Daten Du arbeiten möchtest und welcher Task gelernt werden soll.
    
2. Diskutiere die Idee mit dem Fachcoach.
    
</div>


### Daten und Task
Pytorch stellt einige Datensets zur Verfügung [datasets torch](https://pytorch.org/vision/main/datasets.html).
Verschiedene Kategorien stehen zur Auswahl:
- Image classification
- Image detection or segmentation
- Optical Flow
- Stereo Matching
- Image pairs
- Image captioning
- video classification
- Base classes for custom datasets


**Datenset**  
Eine beliebtes Dateset ist CIFAR10. Es beinhaltet Bilder von 10 Klassen (Flugzeuge, Katzen, Vögel, etc.), die Bilder kommen mit einer Auflösung von 32x32x3 pixel (rgb). Viele Tutorials starten mit diesem Datenset, das lässt darauf schliessen, dass der Rechenaufwand für die Hardware in einem vernümpftigen Rahmen liegt. Daher wird CIFAR10 als Datenset für die Challenge gewählt.

**Task**  
Anhand von CIFAR10 soll ein Modell erstellt werden, welches die Klasse eines Bildes korrekt klassifiziert.



![](cifar10.png)

# Schritt 2: Daten Kennenlernen

<div class="alert alert-block alert-info">
    
1. Mache Dich mit dem Datensatz vertraut, indem Du eine explorative Analyse der Features durchführst: z.B. Vergleich der Klassen pro Feature, Balanciertheit der Klassen. 
2. Führe ein geeignetes Preprocessing durch, z.B. Normalisierung der Daten.  
    
</div>

### 1. Explorative Datenanalyse
[Tutorial Dataloader](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)

In [ ]:
# Laden der Daten von CIFAR10
data_path = './data/'
train_data = torchvision.datasets.CIFAR10(data_path, train=True, download=True)

test_data = torchvision.datasets.CIFAR10(data_path, train=False, download=True)

Datengrösse der Trainings- und Testdaten

In [ ]:
print(f'Anzahl Trainingsdaten: {len(train_data)}\n'
      f'Anzahl Testdaten: {len(test_data)}')

**Wie sind die Bilder im Datensatz gespeichert?**  
Die Bilder sind direkt auf dem Datenset via dem Index abrufbar. Ein Tupel mit dem Bild (RGB, 32x32 pixel) und dem Label (6).

In [ ]:
img_0, label_0 = train_data[0]
img_0, label_0

In [ ]:
# deffinieren der Labels CIFAR10
labels_cifar10_dict = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck",
}
labels_cifar10_list = list(labels_cifar10_dict.values())

**Visualisierung der Bilder**  
Die Bilder können mit matplotlib und imshow() direkt vom Dateset über den index dargestellt werden.


In [ ]:
figure = plt.figure(figsize=(6,6))
cols, rows = 5, 5

for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train_data), size=(1,)).item()
    img, label = train_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_cifar10_list[label], fontsize=8)
    plt.axis('off')
    plt.imshow(img)
plt.show()

**Untersuchen der Verteilungen der Klassen**  
Folgend werden die Verteilungen der Klassen der Cifar10 Datensets auf den Trainings- und Testdaten geprüft.

In [ ]:
train_target = train_data.targets
test_target = test_data.targets
print(f'Labels in Trainingsdaten: {len(train_target)}')
print(f'Labels in Testdaten: {len(test_target)}')

In [ ]:
figure, ax = plt.subplots(1,2, figsize=(10, 3))

ax[0].hist(train_target)
ax[0].set_xticks(np.arange(10))
ax[0].set_xticklabels(labels_cifar10_list, rotation=45)
ax[0].set_title('Trainingset', fontsize=8)

ax[1].hist(test_target)
ax[1].set_xticks(np.arange(10))
ax[1].set_xticklabels(labels_cifar10_list, rotation=45)
ax[1].set_title('Testset', fontsize=8)

plt.suptitle('Verteilung der Klassen von Cifar-10', fontsize=12)
plt.show()

<div class="alert alert-block alert-success">
    
Das Trainingsset umfasst total 50'000 Bilder, davon sind jeweils 5000 Bilder jeder Klasse enthalten (Balanced Datenset). Somit kann zum Beispiel eine Metrik wie'Accuracy' verwendet werden, um die Klassifikation der Modelle zu beurteilen und zu vergleichen. Die 10'000 Bilder in den Testdaten sind ebenfalls gleich verteilt.
    
</div>

Mittelwerte und Standardabweichungen des Trainingdatensets (RGB):
Benötigt um die Daten zu normalisieren

In [ ]:
# Erstelle Datenset mit Tensoren für Berechnunen
transform = transforms.Compose([transforms.ToTensor()])
train_data = torchvision.datasets.CIFAR10(data_path, train=True, download=False, transform=transform)
test_data = torchvision.datasets.CIFAR10(data_path, train=False, download=False, transform=transform)

In [ ]:
def calc_mean_std_dataset(train_data, print_info=True):
    '''
    Berechnet den Mittelwert und Standardabweichung für alle Billder
    '''
    num_images = len(train_data)

    # erstelle Array mit Dimension der input Bilder
    pixel_values = np.zeros((num_images, 3, 32, 32), dtype=np.float32)

    # füllen des Arrays mit Pixel Werten
    for i in range(num_images):
        image, _ = train_data[i]
        pixel_values[i] = image.numpy()

    mean = np.mean(pixel_values, axis=(0, 2, 3))
    std = np.std(pixel_values, axis=(0, 2, 3))

    if print_info:
        print("RGB-Mittelwerte:", mean)
        print("RGB-Standardabweichungen:", std)

calc_mean_std_dataset(train_data, True)

### 2. Preprocessing der Daten  
Folgend werden die Daten in einem Preprocessing Schritt für die Modelle vorbereitet:

In [ ]:
# function preprocessing
def preprocessing_cifar10(path='./data/', train_batch_size=32, test_batch_size=32, normalize='zero_one',
                          norm_mean=(0.4914009  , 0.548215896, 0.4465308), 
                          norm_std=(0.24703279 , 0.24348423 , 0.26158753), 
                          download=True, print_info=False):
    '''
    '''
    if print_info: print(f'------------'), print(f'Preprocessing start')

    if normalize == 'zero_one':
        # transform tensor to normalized range [0, 1], 0=schwarz
        transform = transforms.Compose([transforms.ToTensor()])
    elif normalize == 'minusone_one':
        # transform tensor to normalized range [-1, 1], 0=schwarz
        transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(norm_mean, norm_std)])
    else:
        raise ValueError('Nomalize must be either "zero_one" or "minusone_one"') 
    # if print_info: print('Normalized Tensor'), print(f'mean: {norm_mean} | std: {norm_std}')
    
    # CIFAR10: 50000 32x32 color images in 10 classes, with 5000 images per class
    train_dataset = torchvision.datasets.CIFAR10(root=path, train=True,
                                            download=download, transform=transform)
    test_dataset = torchvision.datasets.CIFAR10(root=path, train=False,
                                        download=download, transform=transform)
    if print_info: 
        print(f'Data transformed: {normalize}')
        calc_mean_std_dataset(train_dataset, print_info)

    # dataloader
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

    if print_info: print(f'Dataloader created with {train_batch_size=}, {test_batch_size=}')
    if print_info: print(f'Preprocessing done'), print('------------')   

    return train_dataset, test_dataset, train_loader, test_loader

In [ ]:
# Hyper-parameters 
data_path = './data/'
batch_size = 32

train_dataset, test_dataset, train_loader, test_loader = preprocessing_cifar10(path=data_path, 
                                                                               train_batch_size=batch_size,
                                                                               test_batch_size=batch_size,
                                                                               normalize='zero_one',  # minusone_one
                                                                               download=False,  
                                                                               print_info=True)

<div class="alert alert-block alert-success">

Aus den Berechnungen des Mittelwert und Standardabweichung lässt sich schliessen dass die Daten von Cifar-10 bereits normalisiert wurden auf Werte zwischen [0,1]. Mit der Transformation im Preprocessing `transforms.Normalize(mean, std)` können die Werte in den Bereich [-1, 1] gesetzt werden. Welcher Wertebereich die bessere Wahl ist, ist Situationsbedingt (z.B. verwendete Aktivierungsfunktion). Eine Zentrierung um Null kann für Netze Vorteile haben, hingegen sind Werte zwischen [0,1] besser zu interpretieren (0=schwarz, 1=RGB 255). Vorerst soll mit dem Wertebereich [0,1] gearbeitet werden.

</div>

# Schritt 3: Aufbau Modellierung  

<div class="alert alert-block alert-info">

1. Lege fest, wie (mit welchen Metriken) Du die Modelle evaluieren möchtest. Berücksichtige auch den Fehler in der Schätzung dieser Metriken.
    
</div>

<div class="alert alert-block alert-success">

Die Klassengrössen sind ausbalanciert, daher ist für die Metrik **Accuracy** für die Modell Beurteilung geeignet und soll hier für die Modellbewertung zum Einsatz kommen:

$$ Accuracy = \frac{Anzahl\ korrekte\ Klassifizierungen}{Total\ Klassifizierungen}  $$

Für eine Klassifizierung von mehr als zwei Klassen eignet sich **CrossEntropyLoss**. Mehr dazu [hier](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html):

$$ \text{CrossEntropyLoss} = -\sum_{i=1}^{N} \left[ y_i \log(\hat{y_i}) + (1 - y_i) \log(1 - \hat{y_i}) \right] $$


Die Accuracy berechnet eine Schätzung. Bei der Initialisierung der Modellgewichte werden Zufallswerte verwendet. Auch die Wahl der Bilder innerhalb der Batchsize wird durch `shuffle=True` zufällig getroffen *(siehe `Preprocessing`)*. Somit varriert die Accuracy nach jedem Modeltraining ein wenig. Cross-Validation würde sich hier anbieten, um auf k-folds unterschiedliche Modelle zu erstellen. Auch ein Modell mehrmals ausführen wäre denkbar. Mit dem berechneten Mittelwert $\mu$ und der Standardabweichung $\sigma$ kann ein Fehlerabschätzung gemacht werde.  
$$ Fehler_{range} = [\mu - \sigma; \mu + \sigma]$$

</div>

<div class="alert alert-block alert-info">

2. Implementiere Basisfunktionalität, um Modelle zu trainieren und gegeneinander zu evaluieren. Wie sollen die Gewichte initialisiert werden?
    
</div>

<div class="alert alert-block alert-success">

Die Methode wie die Intitialisierung der Gewichte stattfindet hat Einfluss wie schnell das Modell konvergiert und hilft die Probleme wie `vanishing` oder `exploding` Gradienten abzuschwächen. Kleine zufällige Werte führen zu einem effizienteren Training. Grosse Werte führen zu Problemenn bei dem das Modell nicht oder nur sehr langsam konvergiert. Je nach Problemstellung und Modelarchitekture können unterschiedliche Initialisierungsmethoden verwendet werden

Mit der Verwendung von `nn.init` (Pytorch) stehen zum Beispiel folgende Optionen zur Verfügung:
1. **Uniform initialization** (help prevents: vanishing gradient, can suffer: exploding gradient)
1. **Xavier initialization** (help prevent: vanishing gradient)
1. **Kaiming initialization** (help prevent: vanishing gradient, account activation function)
1. **Normal initialization** (help prevent: exploding gradient)
1. **Zeros initialization** (can suffer: slow converge, vanishing gradient)
1. **One’s initialization** (can suffer: slow converge, vanishing gradient)

Die genaue Beschreibung der Intitialisierungen können [hier](https://www.geeksforgeeks.org/initialize-weights-in-pytorch/) gefunden werden. Eine eigene Custom-Option stellt Pytorch auch zur Verfügung.

</div>

### Helper Functions  
Um die Erkentnisse dieses Notebook reproduzierbar zu machen, ist es wichtig einen Seed zu definieren. Folgend eine Seed Funktion mit einem üblichen Standart [Quelle](https://vandurajan91.medium.com/random-seeds-and-reproducible-results-in-pytorch-211620301eba).

In [ ]:
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
set_seed()

def plot_loss_epoch(num_epochs, loss, figsize=(8, 4)):
    figure = plt.figure(figsize=figsize)
    plt.plot(np.arange(num_epochs), loss)
    plt.xticks(np.arange(num_epochs))
    plt.title('Loss Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()

def calc_true_predictions(output_model, true_labels):
    _, predicted = torch.max(output_model.data, 1)
    return (predicted == true_labels).sum().item()

def measure_model_time(start_time, calc='min'):
    model_time = np.round(((time.time() - start_time) / 60), 2)  # from seconds to minute
    return model_time

def play_sound(typ=0):
    # play 'finish' sound
    if typ==0:
        winsound.PlaySound('../01_Dokumentation/win_sounds/beep.wav', winsound.SND_ASYNC)
    if typ==1:
        winsound.PlaySound('../01_Dokumentation/win_sounds/beep2.wav', winsound.SND_ASYNC)

def plot_init_weights(model, figsize=(6, 3)):
    l1_weights = model.state_dict()['linear1.weight'].numpy()
    l2_weights = model.state_dict()['linear2.weight'].numpy()
    fig, ax = plt.subplots(1,2, figsize=figsize)

    ax[0].hist(l1_weights.flatten(), bins=50)
    ax[1].hist(l2_weights.flatten(), bins=50)

    ax[0].set_title("Layer 1", fontsize=8)
    ax[0].set_xlabel("Gewichtswert", fontsize=6)
    ax[0].set_ylabel("Anzahl", fontsize=8)
    ax[0].tick_params(axis='y', labelsize=6)
    ax[0].tick_params(axis='x', labelsize=6)
    ax[1].set_title("Layer 2", fontsize=8)
    ax[1].set_xlabel("Gewichtswert", fontsize=6)
    ax[1].tick_params(axis='y', labelsize=6)
    ax[1].tick_params(axis='x', labelsize=6)
    plt.suptitle(f'Verteilung Initialisierungs Methode')
    plt.show()

def test_eval_plot():
    num_epochs = 10
    n_loss_epochs = (0.05 * np.sqrt(np.arange(10))) * -1
    n_correct_train = 0.1 * n_loss_epochs**4
    n_correct_test = 0.05 * n_loss_epochs**4
    # create dataloader
    train_dataset, test_dataset, train_loader, test_loader = preprocessing_cifar10(batch_size=4,
                                                                                norm_mean=(0.5, 0.5, 0.5), 
                                                                                norm_std=(0.5, 0.5, 0.5),
                                                                                download=False,  
                                                                                print_info=False)
    eval_model(num_epochs, n_loss_epochs, n_correct_train, n_correct_test,
                train_loader, test_loader)
# test_eval_plot()   

### Definitionen Evaluation

In [ ]:
def eval_model(num_epochs, n_loss_epochs, n_val_loss_epochs, 
                n_acc_train_epochs, n_acc_test_epochs, train_loader, test_loader, 
                n_loss_batches=None, group_name='group_name', tag_name='tag_name', figsize=(8,5),
                print_info=True, save_image=False):
    
    epoch = np.arange(num_epochs)

    fig, ax1 = plt.subplots(figsize=figsize)
    ax2 = ax1.twinx()

    ax1.plot(epoch, n_loss_epochs, color= 'grey', linestyle='--', label='Train Loss')
    ax1.plot(epoch, n_val_loss_epochs, color= 'grey', label='Test Loss')
    ax2.plot(epoch, n_acc_train_epochs, color='steelblue', label='Accuracy Train')
    ax2.plot(epoch, n_acc_test_epochs, color='coral', label='Accuracy Test')
    
    ax1.set_xticks(epoch)
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss') 
    ax2.set_ylabel('Accuracy')
    ax1.tick_params(axis='x', labelsize=8)
    ax1.tick_params(axis='y', labelsize=8)
    ax2.tick_params(axis='y', labelsize=8)
    ax1.legend(loc='upper center', bbox_to_anchor=(0.25, -0.12), ncol=3)    
    ax2.legend(loc='upper center', bbox_to_anchor=(0.72, -0.12), ncol=3)
    
    plt.suptitle(f'Training: {group_name} ')
    cur_date = datetime.now().strftime("%d.%m.%Y_%H%M")
    plt.title(f'{cur_date} - Tags: {tag_name}', size=6)
    plt.grid()
    if save_image: 
            plt.savefig(f'./plots/eval_model_plots/{cur_date}_{group_name}_{tag_name}.png')
    plt.show()

    if print_info:
        print(f'Accuracy Train {n_acc_train_epochs[-1]:2f}')
        print(f'Accuracy Test {n_acc_test_epochs[-1]:4f}')
        print(f'Loss Train {n_loss_epochs[-1]:2f}')
        print(f'Loss Test {n_val_loss_epochs[-1]:2f}')

    return

### Definitionen Training 

In [ ]:
from datetime import datetime

# Hyperparameters example
config = {
    "name": "MLP", 
    "epochs": 20,   
    "train_batch_size": 32, 
    "test_batch_size": 32,
    "dataset": "CIFAR-10",
    "lr": 1e-3, 
    "optimizer": 'SGD',
    "loss_func": 'CrossEntropyLoss',
    "activation": "ReLU",
    "image_size": 32,
    "cross_validation": False,
    "is_test_batch": True,
    "start_time": datetime.now().strftime("%d.%m.%Y_%H%M"),
    "num_workers": 0,
    "normalize":"zero_one",
    "init_w_method":"kaiming_norm",  #['uniform', 'xavier', 'normal' ,'kaiming_norm' ]
    "hidden_layer_sizes": [128], 
    "kernel_sizes": [],
    "pool_sizes": [],
    "dropout": 0,
    "norm_mean": (0.5, 0.5, 0.5),
    "norm_std": (0.5, 0.5, 0.5),
    "num_classes": 10,
    "save_eval_image": True
}


# Train model function
def train_model(
        model, train_loader, test_loader, config, tags=['tag_name'], group='group_name',
        print_info=False, plot_eval=True, sound=False, write_wandb=True):  
    
    set_seed()

    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
    if print_info: print(f'------------'), print(f'Starten des Trainings auf Device {device}')

    # Model to device
    model = model.to(device)
    model.train()

    # define Loss and Optimizer
    if config['loss_func'] == 'CrossEntropyLoss':
        criterion = nn.CrossEntropyLoss()
        if print_info: print(f"Loss Funktion: {config['loss_func']}")        
    if config['optimizer'] == 'SGD':        
        optimizer = torch.optim.SGD(model.parameters(), lr=config['lr'])
        if print_info: print(f"Optimizer: {config['optimizer']} mit lr: {config['lr']}")
    elif config['optimizer']  == 'Adam':        
        optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
        if print_info: print(f"Optimizer: {config['optimizer']} mit lr: {config['lr']}")

    # Initialize wandb
    if write_wandb: 
        wandb.init(
            project="del-mc1",
            entity='manuel-schwarz',
            group=group,
            name=f"{config['name']}-{config['epochs']}-epochs-{config['optimizer']}-optimizer-{config['start_time']}",
            tags=tags + (['is_test_batch'] if config['is_test_batch'] else []),
            config=config
        )

    # --------------------- train loop --------------------------
    if write_wandb:
        wandb.watch(model)

    n_loss_epochs = []
    n_val_loss_epochs = []
    n_loss_all_batches = []
    n_val_loss_all_batches = []
    n_correct_train_epochs = []
    n_correct_test_epochs = []
    n_acc_train_epochs = []
    n_acc_test_epochs = []

    best_train_loss = 0
    best_val_loss = 0
    best_train_acc = 0
    best_test_acc = 0
    best_epoch = 0

    loop = range(config["epochs"])
    epoch_loop = tqdm(loop, desc="Epochs", position=0, leave=True)

    for epoch in epoch_loop:
        true_train = 0
        n_loss_batch = []

        for i, (images, labels) in enumerate(train_loader):
            if config['is_test_batch'] and i > 1:
                break
            # origin shape: [4, 3, 32, 32] = 4, 3, 1024
            # input_layer: 3 input channels, 6 output channels, 5 kernel size
            images, labels = images.to(device), labels.to(device)

            # Backward and optimize
            optimizer.zero_grad()
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            n_loss_batch.append(loss.item())
            true_train += calc_true_predictions(outputs, labels)
            # acc_train = true_train / len(train_loader.dataset)
            n_loss_all_batches.append(loss.item())

            inner_progress = f"{i+1}/{len(train_loader)}"
            epoch_loop.set_description(f"Epochs (Batch: {inner_progress})")
            epoch_loop.refresh()

        # save results
        acc_train = true_train / len(train_loader.dataset)
        epoch_loss = np.mean(n_loss_batch)

        n_correct_train_epochs.append(true_train)        
        n_acc_train_epochs.append(acc_train)  
        n_loss_epochs.append(epoch_loss)

        # ---------- eval
        model.eval()
        true_test_val = 0 
        n_val_loss_batch = []
        # n_val_pred = []

        with torch.no_grad():
            for i, (images, labels) in enumerate(test_loader):
                if config['is_test_batch'] and i > 1:
                    break
                images, labels = images.to(device), labels.to(device)
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

                n_val_loss_batch.append(loss.item())
                true_test_val += calc_true_predictions(outputs, labels)
                n_val_loss_all_batches.append(loss.item())              
                # _, predicted = torch.max(outputs, 1)  # prediction speichern todo

        # save results
        acc_test = true_test_val / len(test_loader.dataset)
        epoch_val_loss = np.mean(n_val_loss_batch)

        n_correct_test_epochs.append(true_test_val)
        n_acc_test_epochs.append(acc_test) 
        n_val_loss_epochs.append(epoch_val_loss)

        model.train()

        # best scores
        if epoch_loss < best_train_loss:
            best_train_loss = epoch_loss

        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss

        if acc_train > best_train_acc:
            best_train_acc = acc_train

        if acc_test > best_test_acc:
            best_test_acc = acc_test
            best_epoch = epoch

        # log metrics to wandb
        if write_wandb:
            wandb.log({
                "loss epoch": epoch_loss, 
                "validation loss epoch": epoch_val_loss, 
                "acc_train": acc_train, 
                "acc_test": acc_test,
                "best_epoch": best_epoch,
                "best_epoch": best_train_loss,
                "best_epoch": best_val_loss,
                "best_epoch": best_train_acc,
                "best_epoch": best_test_acc,
                })

    acc_train = n_correct_train_epochs[-1] / len(train_loader.dataset)
    acc_test = n_correct_test_epochs[-1] / len(test_loader.dataset)

    if plot_eval:
        eval_model(config['epochs'], n_loss_epochs, n_val_loss_epochs, 
                    n_acc_train_epochs, n_acc_test_epochs, train_loader, test_loader,
                    group_name=group, tag_name=tags, print_info=print_info, save_image=config['save_eval_image'])
        
    print(
            f"Epoch {epoch + 1}/{config['epochs']}, \
            Loss: {round(epoch_loss, 5)}, Validation Loss: {round(epoch_val_loss, 5)} \
            Acc: {round(acc_train, 5)}, Validation Acc: {round(acc_test, 5)}"
            )
    
    if sound: play_sound(1)

    if write_wandb: 
        wandb.finish()

    print('Finished Training')
    return epoch_loss, epoch_val_loss, acc_train, acc_test


### Einfacher Modell Test  
Folgend soll ein einfaches Modell mit einem hidden Layer erstellt werden um die Basisfunktionen von `train_model()` zu testen. 

Initilisierung Gewichte: 
Da unterschiedliche Problemstellung verschiedene Initialisierungen erfordern, sollen mehrere Methoden ausprobiert werden um die Gewichte zu initialisiern `linear_layer = torch.nn.Linear(2, 3)`:
1. `uniform`: torch.nn.init.uniform_(linear_layer.weight)
1. `xavier`: torch.nn.init.xavier_uniform_(linear_layer.weight)
1. `normal`: torch.nn.init.normal_(linear_layer.weight, mean=0, std=1)
1. `kaiming_norm`: torch.nn.init.kaiming_normal_(linear_layer.weight, nonlinearity="relu")


In [ ]:
# Modell Class
class MLPNet_hl1(nn.Module):
    def __init__(self, hidden_l_1:list, act_fn=F.relu, dropout=0, init_methode:str='kaiming_norm'):
        super(MLPNet_hl1, self).__init__()
        self.linear1 = nn.Linear(3*32*32, hidden_l_1[0])  # input.shape = (n, 3, 32, 32)
        self.linear2 = nn.Linear(hidden_l_1[0], 10)
        self.dropout = nn.Dropout(dropout)
        self.activation = act_fn 

        for linear_weight in [self.linear1.weight, self.linear2.weight]:          
            if init_methode == 'uniform':
                torch.nn.init.uniform_(linear_weight)
            if init_methode == 'xavier':
                torch.nn.init.xavier_uniform_(linear_weight)
            if init_methode == 'normal':
                torch.nn.init.normal_(linear_weight, mean=0, std=1)
            if init_methode == 'kaiming_norm':
                torch.nn.init.kaiming_normal_(linear_weight, nonlinearity='relu')

    def forward(self, x): # x.shape = (n, 3, 32, 32)
        x = x.view(-1, 3*32*32) # x.shape = (n, 3072)
        x = self.activation(self.linear1(x)) # x.shape = (n, 128)
        x = self.dropout(x)
        x = self.linear2(x) # x.shape = (n, 10)
        return x

In [ ]:
method_lst = ['uniform', 'xavier', 'normal' ,'kaiming_norm' ]
# create model
for method in method_lst:
    model = MLPNet_hl1(hidden_l_1=[64], init_methode=method)
    plot_init_weights(model, figsize=(3, 1))

Die Verteilungen der Gewichte mit den Methoden `uniform` und  `xavier` sind sehr ähnlich. Mit Xavier ist die X-Skala unterschiedlich, da die Gewichte so skalliert werden dass die Varianz des Output der Varianz des Inputs entspricht.  
Bei `normal` und `kaiming_norm` besteht das gleiche Prinzip, die skallierung der Gewichte, zudem wird die Aktivierungsfunktion berücksichtigt.

In [ ]:
if True:
    # Hyperparameters  
    config = {
        "name": "MLP-test1", 
        "epochs": 5,   
        "train_batch_size": 32, 
        "test_batch_size": 32,
        "dataset": "CIFAR-10",
        "lr": 1e-3, 
        "optimizer": 'SGD',
        "loss_func": 'CrossEntropyLoss',
        "activation": "ReLU",
        "image_size": 32,
        "cross_validation": False,
        "is_test_batch": False,
        "start_time": datetime.now().strftime("%d.%m.%Y_%H%M"),
        "num_workers": 0,
        "normalize":"zero_one",
        "init_w_method":"kaiming_norm",  #['uniform', 'xavier', 'normal' ,'kaiming_norm' ]
        "hidden_layer_sizes": [128], 
        "kernel_sizes": [],
        "pool_sizes": [],
        "dropout": 0,
        "norm_mean": (0.5, 0.5, 0.5),
        "norm_std": (0.5, 0.5, 0.5),
        "num_classes": 10,
        "save_eval_image": True
    }

    # create dataloader
    _, _, train_loader, test_loader = preprocessing_cifar10(train_batch_size=config["train_batch_size"],
                                                            test_batch_size=config["test_batch_size"],
                                                            normalize=config['normalize'],
                                                            download=False,  
                                                            print_info=False)
    # create model
    MLPNet_hl1(hidden_l_1=config["hidden_layer_sizes"], 
            init_methode=config["init_w_method"])

    # train model
    train_model(model, train_loader, test_loader, config=config,
                group='MLP_test',
                tags=['MLP'],
                print_info=False, plot_eval=True, sound=True, 
                write_wandb=True
                )


# Schritt 4: Evaluation  

<div class="alert alert-block alert-warning">

Bei der Evaluation ist darauf zu achten, dass das Vorgehen stets möglichst reflektiert erfolgt und versucht wird, die Ergebnisse zu interpretieren. Am Schluss soll auch ein Fazit gezogen werden, darüber welche Variante am besten funktioniert.  

</div>

<div class="alert alert-block alert-info">

**1. Training mit SGD, ohne REG, ohne BN:**  
Untersuche verschiedene Modelle unterschiedlicher Komplexität, welche geeignet sein könnten, um das Klassifikationsproblem zu lösen. Verwende Stochastic Gradient Descent - ohne Beschleunigung, ohne Regularisierung (REG) und ohne Batchnorm (BN).  

a. Für jedes Modell mit gegebener Anzahl Layer und Units pro Layer führe ein sorgfältiges Hyper-Parameter-Tuning durch (Lernrate, Batch-Grösse). Achte stets darauf, dass das Training stabil läuft. Merke Dir bei jedem Training, den Loss, die Performance Metrik(en) inkl. Schätzfehler, die verwendete Anzahl Epochen, Lernrate und Batch-Grösse.

b. Variiere die Anzahl Layer und Anzahl Units pro Layer, um eine möglichst gute Performance zu erreichen. Falls auch CNNs (ohne Transfer-Learning) verwendet werden variiere auch Anzahl Filter, Kernel-Grösse, Stride, Padding.

c. Fasse die Ergebnisse zusammen in einem geeigneten Plot, bilde eine Synthese und folgere, welche Modell-Komplexität Dir am sinnvollsten erscheint.  

</div>


### Deffinition von MLP Modellen

In [ ]:
# Singel MLP Model class
class MLPNet_hlXX(nn.Module):
    def __init__(self, hidden_l_1:list, act_fn=F.relu, dropout=0, init_methode:str='kaiming_norm'):
        super(MLPNet_hlXX, self).__init__()
        self.init_methode = init_methode
        self.num_h_layers = len(hidden_l_1)
        print(f'number Layers: {self.num_h_layers}')

        self.linear1 = nn.Linear(3*32*32, hidden_l_1[0])  # input.shape = (n, 3, 32, 32)
        if self.num_h_layers == 1:            
            self.linear1.weight = self.init_dits_weight(self.linear1.weight)
            self.linear2 = nn.Linear(hidden_l_1[0], 10)
            self.linear2.weight = self.init_dits_weight(self.linear2.weight)
            self.dropout = nn.Dropout(dropout)
            self.activation = act_fn 
            
        if self.num_h_layers == 2:
            self.linear1.weight = self.init_dits_weight(self.linear1.weight)
            self.linear2 = nn.Linear(hidden_l_1[0], hidden_l_1[1])
            self.linear2.weight = self.init_dits_weight(self.linear2.weight)
            self.linear3 = nn.Linear(hidden_l_1[1], 10)
            self.linear3.weight = self.init_dits_weight(self.linear3.weight)
            self.dropout = nn.Dropout(dropout)
            self.activation = act_fn 

        if self.num_h_layers == 3:
            self.linear1.weight = self.init_dits_weight(self.linear1.weight)
            self.linear2 = nn.Linear(hidden_l_1[0], hidden_l_1[1])
            self.linear2.weight = self.init_dits_weight(self.linear2.weight)
            self.linear3 = nn.Linear(hidden_l_1[1], hidden_l_1[2])
            self.linear3.weight = self.init_dits_weight(self.linear3.weight)
            self.linear4 = nn.Linear(hidden_l_1[2], 10)
            self.linear4.weight = self.init_dits_weight(self.linear3.weight)
            self.dropout = nn.Dropout(dropout)
            self.activation = act_fn 

        if self.num_h_layers == 4:
            self.linear1.weight = self.init_dits_weight(self.linear1.weight)
            self.linear2 = nn.Linear(hidden_l_1[0], hidden_l_1[1])
            self.linear2.weight = self.init_dits_weight(self.linear2.weight)
            self.linear3 = nn.Linear(hidden_l_1[1], hidden_l_1[2])
            self.linear3.weight = self.init_dits_weight(self.linear3.weight)
            self.linear3 = nn.Linear(hidden_l_1[2], hidden_l_1[3])
            self.linear3.weight = self.init_dits_weight(self.linear3.weight)
            self.linear4 = nn.Linear(hidden_l_1[3], 10)
            self.linear4.weight = self.init_dits_weight(self.linear3.weight)
            self.dropout = nn.Dropout(dropout)
            self.activation = act_fn 

    def forward(self, x): # x.shape = (n, 3, 32, 32)
        x = x.view(-1, 3*32*32) # x.shape = (n, 3072)
        if self.num_h_layers == 1:            
            x = self.activation(self.linear1(x)) # x.shape = (n, 128)
            x = self.dropout(x)
            x = self.linear2(x) # x.shape = (n, 10)

        if self.num_h_layers == 2:
            x = self.activation(self.linear1(x)) # x.shape = (n, 128)
            x = self.dropout(x)
            x = self.activation(self.linear2(x))
            x = self.dropout(x)
            x = self.linear3(x) # x.shape = (n, 10)

        if self.num_h_layers == 3:
            x = self.activation(self.linear1(x)) # x.shape = (n, 128)
            x = self.dropout(x)
            x = self.activation(self.linear2(x))
            x = self.dropout(x)
            x = self.activation(self.linear3(x))
            x = self.dropout(x)
            x = self.linear4(x) # x.shape = (n, 10)

        if self.num_h_layers == 4:
            x = x.view(-1, 3*32*32) # x.shape = (n, 3072)
            x = self.activation(self.linear1(x)) # x.shape = (n, 128)
            x = self.dropout(x)
            x = self.activation(self.linear2(x))
            x = self.dropout(x)
            x = self.activation(self.linear3(x))
            x = self.dropout(x)
            x = self.activation(self.linear4(x))
            x = self.dropout(x)
            x = self.linear5(x) # x.shape = (n, 10)    
        return x
    
    def init_dits_weight(self, linear_weight):
        if self.init_methode == 'uniform':
            return torch.nn.init.uniform_(linear_weight)
        if self.init_methode == 'xavier':
            return torch.nn.init.xavier_uniform_(linear_weight)
        if self.init_methode == 'normal':
            return torch.nn.init.normal_(linear_weight, mean=0, std=1)
        if self.init_methode == 'kaiming_norm':
            return torch.nn.init.kaiming_normal_(linear_weight, nonlinearity='relu')

### Deffinition von CNN Modellen

In [ ]:
# Modell Class
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 5 * 5)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x

### Deffinition wandb Trainingloop

In [ ]:
# def train_mlp_1hl():
#         wandb.init(settings=wandb.Settings(silent="True"))
#         config = wandb.config
#         run_name = f'{config["name"]}_lr_{str(config["lr"])}_layer_{str(config["hidden_layer_sizes"])}_bs_{str(config["batchsize"])}'
#         print(run_name)
#         wandb.run.name = run_name

#         # create dataloader
#         _, _, train_loader, test_loader = preprocessing_cifar10(batch_size=config["batchsize"],
#                                                                                         norm_mean=config["norm_mean"], 
#                                                                                         norm_std=config["norm_std"],
#                                                                                         download=False,  
#                                                                                         print_info=False)
#         # create model
#         MLPNet_hl1(hidden_l_1=config["hidden_layer_sizes"][0], 
#                 init_methode=config["init_w_method"])

#         # train model
#         train_model(model, train_loader, test_loader,
#                         loss_func=config["loss_func"], 
#                         opt=config["opt"], 
#                         lr=config["lr"], 
#                         num_epochs=config["num_epochs"], 
#                         id_name=config["name"],
#                         print_info=False, plot_eval=False, sound=False, write_wandb=True)
        
def train_mlp_hlXX():
        wandb.init(settings=wandb.Settings(silent="True"))
        config = wandb.config
        run_name = f'{config["name"]}_lr_{str(config["lr"])}_layer_{str(config["hidden_layer_sizes"])}_bs_{str(config["batchsize"])}'
        print(run_name)
        wandb.run.name = run_name

        # create dataloader
        _, _, train_loader, test_loader = preprocessing_cifar10(batch_size=config["batchsize"],
                                                                                        norm_mean=config["norm_mean"], 
                                                                                        norm_std=config["norm_std"],
                                                                                        download=False,  
                                                                                        print_info=False)
        # create model
        MLPNet_hlXX(hidden_l_1=config["hidden_layer_sizes"], 
                init_methode=config["init_w_method"])

        # train model
        train_model(model, train_loader, test_loader,
                        loss_func=config["loss_func"], 
                        opt=config["opt"], 
                        lr=config["lr"], 
                        num_epochs=config["num_epochs"], 
                        tags=config["name"],
                        print_info=True, plot_eval=False, sound=False, write_wandb=True)


Test modellklassen

In [ ]:
# train_simple_model = False
# if train_simple_model:
#     send_data_to_wand = False
#     exp_name = "simple mlp (1hl)"
#     project_name = "del_mc1_test"

#     #--------- Hyperparameters ---------- 
#     config = {
#         "dataset": "CIFAR-10",
#         "name": exp_name,
#         "architecture": "MLP", 
#         "num_epochs": 2,   
#         "batchsize": 64,
#         "lr": 1e-3, 
#         "opt": 'SGD',
#         "loss_func": 'CrossEntropyLoss',
#         "hidden_layer_sizes": [64, 128, 256], 
#         "kernel_sizes": [],
#         "activation": "ReLU",
#         "pool_sizes": [],
#         "dropout": 0,
#         "norm_mean": (0.5, 0.5, 0.5),
#         "norm_std": (0.5, 0.5, 0.5),
#         "num_classes": 10,
#         "init_w_method":"kaiming_norm"  #['uniform', 'xavier', 'normal' ,'kaiming_norm' ]
#     }

#     if send_data_to_wand:
#         wandb.init(project=project_name, name=exp_name, notes="", config=config)


#     # create dataloader
#     train_dataset, test_dataset, train_loader, test_loader = preprocessing_cifar10(batch_size=config["batchsize"],
#                                                                                 norm_mean=config["norm_mean"], 
#                                                                                 norm_std=config["norm_std"],
#                                                                                 download=False,  
#                                                                                 print_info=False)
#     # create model
#     MLPNet_hlXX(hidden_l_1=config["hidden_layer_sizes"], 
#             init_methode=config["init_w_method"])

#     # train model
#     train_model(model, train_loader, test_loader,
#                 loss_func=config["loss_func"], 
#                 opt=config["opt"], 
#                 lr=config["lr"], 
#                 num_epochs=config["num_epochs"], 
#                 id_name=config["name"],
#                 print_info=True, plot_eval=True, sound=True, write_wandb=send_data_to_wand)


#     # Weights & Biases save Parameter
#     if send_data_to_wand:
#         wandb.finish()

## **MLP** Parameter Suche mit wandb  
Modelltraining Parametereinstellungen mit wandb sweep [Anleitung](https://docs.wandb.ai/ref/python/sweep)

Für ein gutes Modell müssen die Hyperparamter optimal gesetzt sein. Diese optimalen Parameter zu finden ist nicht einfach. Hier soll Schritt für Schritt das MLP-Modell verbessert werden. Folgende Anpassunge sollen ausgesucht werden:  
* Anzahl hidden Layers erweitern
* Umfang der hidden Layers vergrössern
* Anpassung oder hinzufügen von Aktivierungsfunktionen
* Anpassen der Lernrate
* Trainingszeit erhöhen

Um die Übersicht über all die unterschiedlichen Kombinationen von Hyperparameter nicht zu verlieren, wird das Tracking Tool [weights&biases]() mit der `wandb.sweep()` Configurations Funktion verwendet. Damit werden die Parameter schrittweise angepasst und jeweils ein Modell auf dem Dashboard von weights&biases abgebildet. Verschiedene Grafiken stehen danach zur Interpretation der Modell zur Verfügung.  

---
Folgend werden Phasen definiert um den Einfluss von unterschiedliche Kombinationen von Parameter zu testen. Die Parameter werden entsprechend im `sweep_configuration` festgehalten:
1. Phase: Lernraten, Batchgrösse und Anzahl Layer sollen untersucht werden.
1. Phase: Lernraten, Anzahl Layer und Layergrösse sollen untersucht werden
1. Phase: Untersuchung der Initialisierung der Gewichte auf Layer
1. Phase: Änderungen von Aktivierungsfunktionen
1. Phase: Trainingszeiten für vielversprechende Modellparameter erhöhen


**Phase 1:**  
Lernraten, Anzahl Layer sollen untersucht werden.

In [ ]:
train_sweep = False  # cuda:167min
exp_name = "mlp_ph1"
num_epochs = 20

sweep_configuration = {
    "name": exp_name,
    "metric": {"name": "accuracy", "goal": "maximize"},
    "method": "grid",
    "parameters": {"name": {"values": [exp_name]},                    
                    "loss_func": {"values": ['CrossEntropyLoss']},
                    "opt": {"values": ["SGD"]},
                    "init_w_method": {"values": ["kaiming_norm"]},  #['uniform', 'xavier', 'normal' ,'kaiming_norm' ]
                    "norm_mean": {"values": [(0.5, 0.5, 0.5)]},
                    "norm_std": {"values": [(0.5, 0.5, 0.5)]},
                    "num_epochs": {"values": [num_epochs]},
                    "batchsize": {"values": [64, 128, 256, 512]},
                    "hidden_layer_sizes": {"values": [(32,), (32,32), (32,32,32), (32,32,32,32)]},
                    "lr": {"values": [1e-3, 1e-4]}
                    },
}

if train_sweep:
    sweep_id = wandb.sweep(sweep_configuration, project="del_mc1_sweeptest")
    wandb.agent(sweep_id, function=train_mlp_hlXX)
    play_sound(1)

**Phase 2:** 
Lernraten, Anzahl Layer und Layergrösse sollen untersucht werden  
* batchsize fix 64 (grösser als 128 deutet auf Overfitting)
* Layergrössen aufsteigend erweitern

In [ ]:
train_sweep = False  # cuda:112min
exp_name = "mlp_ph2"
num_epochs = 20

sweep_configuration = {
    "name": exp_name,
    "metric": {"name": "accuracy", "goal": "maximize"},
    "method": "grid",
    "parameters": {"name": {"values": [exp_name]},                    
                    "loss_func": {"values": ['CrossEntropyLoss']},
                    "opt": {"values": ["SGD"]},
                    "init_w_method": {"values": ["kaiming_norm"]},  #['uniform', 'xavier', 'normal' ,'kaiming_norm' ]
                    "norm_mean": {"values": [(0.5, 0.5, 0.5)]},
                    "norm_std": {"values": [(0.5, 0.5, 0.5)]},
                    "num_epochs": {"values": [num_epochs]},
                    "batchsize": {"values": [64, 128]},
                    "hidden_layer_sizes": {"values": [(32,64), (64,128), (128,256), (32,64,128), (64,128,256)]},
                    "lr": {"values": [1e-3, 1e-4]}
                    },
}

if train_sweep:
    sweep_id = wandb.sweep(sweep_configuration, project="del_mc1_sweeptest")
    wandb.agent(sweep_id, function=train_mlp_hlXX)
    play_sound(1)

In [ ]:
train_sweep = False
exp_name = "mlp_ph2"
num_epochs = 20

sweep_configuration = {
    "name": exp_name,
    "metric": {"name": "accuracy", "goal": "maximize"},
    "method": "grid",
    "parameters": {"name": {"values": [exp_name]},                    
                    "loss_func": {"values": ['CrossEntropyLoss']},
                    "opt": {"values": ["SGD"]},
                    "init_w_method": {"values": ["kaiming_norm"]},  #['uniform', 'xavier', 'normal' ,'kaiming_norm' ]
                    "norm_mean": {"values": [(0.5, 0.5, 0.5)]},
                    "norm_std": {"values": [(0.5, 0.5, 0.5)]},
                    "num_epochs": {"values": [num_epochs]},
                    "batchsize": {"values": [64]},
                    "hidden_layer_sizes": {"values": [(32,64), (64,128), (128,256), (32,64,128), (64,128,256)]},
                    "lr": {"values": [1e-3, 1e-4]}
                    },
}

if train_sweep:
    sweep_id = wandb.sweep(sweep_configuration, project="del_mc1_sweeptest")
    wandb.agent(sweep_id, function=train_mlp_hlXX)
    play_sound(1)

### **CNN** Parameter Suche mit wandb

<div class="alert alert-block alert-info">

**2. Nutzen der Regularisierung**  
Ziehe nun verschiedene Regularisierungsmethoden bei den MLP Layern in Betracht:  
a. L1/L2 Weight Penalty  
b. Dropout

Evaluiere den Nutzen der Regularisierung, auch unter Berücksichtigung verschiedener Regularisierungsstärken. Beschreibe auch kurz, was allgemein das Ziel von Regularisierungsmethoden ist (Regularisierung im Allgemeinen, sowie auch Idee der einzelnen Methoden). Inwiefern wird dieses Ziel im gegebenen Fall erreicht?
    
</div>

<div class="alert alert-block alert-info">

**3. Nutzen von Batchnorm BN (ohne REG, mit SGD)**  
Evaluiere, ob Batchnorm etwas bringt. Beschreibe kurz, was die Idee von BN ist, wozu es helfen soll.
    
</div>    

<div class="alert alert-block alert-info">

**4. Nutzen von Adam (ohne BN, ohne / mit REG)**   
Evaluiere, ob Du mit Adam bessere Resultate erzielen kannst.
    
</div>    

# Tests

<div class="alert alert-block alert-info">
<b>Tipp:</b> Blaue Boxen .. 
</div>

<div class="alert alert-block alert-success">
<b>Erfolg:</b> Grüne Boxen (alert-success)
</div>

<div class="alert alert-block alert-danger">
<b>Warnung:</b> Rote Boxen (alert-danger)
</div>